# Notebook 4 — Model Selection with Grouped Time-Series Cross-Validation

Purpose:
- Load the feature-engineered dataset (../4_data_analysis/model_datasets/model_ready_dataset_fe.csv)
- Run grouped rolling-origin cross-validation (expanding window per-region), evaluate models across splits
- Compare target transforms (raw vs log1p) for Rainfall
- Run a small randomized hyperparameter search (custom loop) for RandomForest and XGBoost.
- Save best parameters to ../4_data_analysis/model_datasets/best_params.json

Notes:
- This notebook focuses on safe time-aware validation (no leakage across time or regions).
- The CV implemented here builds splits across all regions where each split uses per-region expanding windows and then concatenates indices so the model sees many small series together but never future data.


In [16]:
import os
import pandas as pd
import numpy as np
import json
from math import sqrt
from pprint import pprint
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import random

np.random.seed(42)
random.seed(42)

fe_path = os.path.join('..','4_data_analysis','model_datasets','model_ready_dataset_fe.csv')
if not os.path.exists(fe_path):
    raise FileNotFoundError(f'Feature-engineered dataset not found at {fe_path}. Run Notebook 3 first.')
df = pd.read_csv(fe_path)
print('Loaded FE data shape:', df.shape)


Loaded FE data shape: (2040, 21)


In [17]:
# Basic checks
print('Columns:', df.columns.tolist())
if 'Time' not in df.columns:
    df = df.sort_values(['REGION','YEAR','Month_Num']).reset_index(drop=True)
    df['Time'] = df.groupby('REGION').cumcount()

print('Per-region counts:')
print(df.groupby('REGION').size())


Columns: ['REGION', 'YEAR', 'Month', 'Rainfall', 'Temperature', 'Month_Num', 'Time', 'Rainfall_lag_1', 'Rainfall_lag_2', 'Rainfall_lag_3', 'Rainfall_lag_12', 'Temperature_lag_1', 'Temperature_lag_2', 'Temperature_lag_3', 'Temperature_lag_12', 'Rainfall_roll3', 'Rainfall_roll12', 'Temperature_roll3', 'Temperature_roll12', 'Month_sin', 'Month_cos']
Per-region counts:
REGION
Central    408
East       408
North      408
South      408
West       408
dtype: int64


In [18]:
# Grouped expanding CV builder
def grouped_expanding_splits(df, group_col='REGION', time_col='Time',
                            initial_window=36, step=6, max_splits=5):
    """
    Returns list of (train_idx, val_idx) where indices are global df indices.
    For each region we create expanding windows and then combine splits across regions where all regions have that split available.
    """
    groups = df[group_col].unique()
    per_group_splits = {}
    for g in groups:
        idx = df[df[group_col]==g].sort_values(time_col).index.to_numpy()
        n = len(idx)
        splits = []
        start = initial_window
        while start < n and len(splits) < max_splits:
            train_idx = idx[:start]
            val_idx = idx[start:start+step]
            if len(val_idx) == 0:
                break
            splits.append((train_idx, val_idx))
            start += step
        per_group_splits[g] = splits

    # Combine per-round across groups
    combined = []
    for round_i in range(max_splits):
        train_all = []
        val_all = []
        for g in groups:
            grp_splits = per_group_splits.get(g, [])
            if round_i >= len(grp_splits):
                train_all = None
                break
            tr, va = grp_splits[round_i]
            train_all.extend(tr.tolist())
            val_all.extend(va.tolist())
        if train_all is None:
            break
        combined.append((np.array(sorted(set(train_all))), np.array(sorted(set(val_all)))))
    return combined

# Example: build splits (adjust initial_window/step as needed for the dataset)
splits = grouped_expanding_splits(df, initial_window=36, step=12, max_splits=4)
print('Number of combined splits:', len(splits))
for i, (tr, va) in enumerate(splits):
    print(f'Split {i}: train {len(tr)} rows, val {len(va)} rows')


Number of combined splits: 4
Split 0: train 180 rows, val 60 rows
Split 1: train 240 rows, val 60 rows
Split 2: train 300 rows, val 60 rows
Split 3: train 360 rows, val 60 rows


In [19]:
# Evaluation helpers
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def r2(y_true, y_pred):
    return r2_score(y_true, y_pred)

def evaluate_preds(y_true, y_pred):
    return {
        'rmse': rmse(y_true, y_pred),
        'mae': mae(y_true, y_pred),
        'r2': r2(y_true, y_pred)
    }



We will test three model types and two target variants.
- Models: Ridge (fast baseline), RandomForest (tree), XGBoost 
- Target variants: Raw Rainfall and log1p(Rainfall)

We use a small randomized search (custom loop) rather than GridSearchCV to keep compute limited and to ensure we use our custom CV splits safely.

In [20]:
# Feature set for modeling - choose features that exist in FE dataset
candidate_features = [c for c in df.columns if c not in ['REGION','YEAR','Month','Month_Num','Rainfall','Temperature','Time']]
print('Candidate features sample:', candidate_features[:30])

# We'll include YEAR, Time and the created lags/rolls/cyclical if present
base_features = [c for c in ['YEAR','Time','Month_sin','Month_cos'] if c in df.columns]
lag_features = [c for c in df.columns if c.startswith('Rainfall_lag_') or c.startswith('Temperature_lag_')]
roll_features = [c for c in df.columns if c.endswith('_roll3') or c.endswith('_roll12') or c.endswith('_roll3') or c.endswith('_roll12')]

features = base_features + lag_features + roll_features
features = [f for f in features if f in df.columns]
print('Modeling features count:', len(features))


Candidate features sample: ['Rainfall_lag_1', 'Rainfall_lag_2', 'Rainfall_lag_3', 'Rainfall_lag_12', 'Temperature_lag_1', 'Temperature_lag_2', 'Temperature_lag_3', 'Temperature_lag_12', 'Rainfall_roll3', 'Rainfall_roll12', 'Temperature_roll3', 'Temperature_roll12', 'Month_sin', 'Month_cos']
Modeling features count: 16


In [21]:
# Define simple param search spaces
rf_param_space = {
    'n_estimators': [100, 200, 400],
    'max_depth': [6, 10, 16, None],
    'max_features': [None, 'sqrt', 0.5]
}
xgb_param_space = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.05, 0.1]
}
ridge_param_space = {'alpha': [0.1, 1.0, 10.0]}

# helper to sample one combo from a dict-of-lists
def sample_params(space):
    return {k: random.choice(v) for k,v in space.items()}


In [22]:
# Custom CV evaluation loop (returns average RMSE across splits)
def evaluate_model_cv(model_cls, param_space, df, features, target_col, splits, target_transform=None, n_iter=6):
    results = []
    for it in range(n_iter):
        params = sample_params(param_space)
        rmses = []
        maes = []
        r2s = []
        for tr_idx, va_idx in splits:
            X_tr = df.loc[tr_idx, features]
            X_va = df.loc[va_idx, features]
            y_tr = df.loc[tr_idx, target_col].values
            y_va = df.loc[va_idx, target_col].values
            if target_transform is not None:
                y_tr = target_transform(y_tr)
                
            # instantiate model
            if model_cls == 'ridge':
                model = Ridge(**params)
            elif model_cls == 'rf':
                model = RandomForestRegressor(n_jobs=-1, **params)
            elif model_cls == 'xgb':
                try:
                    from xgboost import XGBRegressor
                except Exception as e:
                    raise ImportError('XGBoost not installed or failed to import: ' + str(e))
                model = XGBRegressor(objective='reg:squarederror', n_jobs=1, **params)
            else:
                raise ValueError('Unknown model class')

            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_va)
            # inverse transform for scoring if transform used
            if target_transform is not None:
                # for log1p we applied y = log1p(y); inverse is expm1
                y_pred_inv = np.expm1(y_pred)
                y_va_inv = np.expm1(y_va)
            else:
                y_pred_inv = y_pred
                y_va_inv = y_va
            rmses.append(mean_squared_error(y_va_inv, y_pred_inv, squared=False))
            maes.append(mean_absolute_error(y_va_inv, y_pred_inv))
            r2s.append(r2_score(y_va_inv, y_pred_inv))
        results.append({'params': params, 'rmse_mean': np.mean(rmses), 'mae_mean': np.mean(maes), 'r2_mean': np.mean(r2s)})
    # return sorted by rmse
    results = sorted(results, key=lambda x: x['rmse_mean'])
    return results


In [ ]:
# Evaluate a few configurations
import warnings
warnings.filterwarnings("ignore")

target = 'Rainfall'
splits_for_eval = splits  # from earlier

print('Running Ridge on raw target...')
ridge_results_raw = evaluate_model_cv('ridge', ridge_param_space, df, features, target, splits_for_eval, target_transform=None, n_iter=3)
pprint(ridge_results_raw[:3])

print('\nRunning Ridge on log1p target...')
ridge_results_log = evaluate_model_cv('ridge', ridge_param_space, df, features, target, splits_for_eval, target_transform=np.log1p, n_iter=3)
pprint(ridge_results_log[:3])

print('\nRunning small RF search on raw target...')
rf_results_raw = evaluate_model_cv('rf', rf_param_space, df, features, target, splits_for_eval, target_transform=None, n_iter=4)
pprint(rf_results_raw[:3])

# Try XGBoost if available
try:
    print('\nRunning XGBoost search on raw target')
    xgb_results = evaluate_model_cv('xgb', xgb_param_space, df, features, target, splits_for_eval, target_transform=None, n_iter=4)
    pprint(xgb_results[:3])
except ImportError as e:
    print('XGBoost not available; skipping XGB evaluation:', e)


Running Ridge on raw target...
[{'mae_mean': 0.45230296009969895,
  'params': {'alpha': 1.0},
  'r2_mean': 0.8497176895038167,
  'rmse_mean': 0.6746728295841031},
 {'mae_mean': 0.45602294830794365,
  'params': {'alpha': 0.1},
  'r2_mean': 0.8484244632651129,
  'rmse_mean': 0.6770691965310607},
 {'mae_mean': 0.45602294830794365,
  'params': {'alpha': 0.1},
  'r2_mean': 0.8484244632651129,
  'rmse_mean': 0.6770691965310607}]

Running Ridge on log1p target...
[{'mae_mean': 53.8187593218649,
  'params': {'alpha': 10.0},
  'r2_mean': -0.03625147664667383,
  'rmse_mean': 255.7463811419369},
 {'mae_mean': 53.83602776169591,
  'params': {'alpha': 1.0},
  'r2_mean': -0.036637997548354506,
  'rmse_mean': 255.77905884334683},
 {'mae_mean': 53.83602776169591,
  'params': {'alpha': 1.0},
  'r2_mean': -0.036637997548354506,
  'rmse_mean': 255.77905884334683}]

Running small RF search on raw target...
[{'mae_mean': 0.35895996287680243,
  'params': {'max_depth': 10, 'max_features': 0.5, 'n_estimators'

In [28]:
# Save best found hyperparameters (examples: top of each results list)
best_params = {}
if ridge_results_raw:
    best_params['ridge_raw'] = ridge_results_raw[0]['params']
if ridge_results_log:
    best_params['ridge_log1p'] = ridge_results_log[0]['params']
if rf_results_raw:
    best_params['rf_raw'] = rf_results_raw[0]['params']
try:
    if xgb_results:
        best_params['xgb_log1p'] = xgb_results[0]['params']
except NameError:
    pass

out_params_path = os.path.join('..','4_data_analysis','model_datasets','best_params.json')
with open(out_params_path, 'w') as f:
    json.dump(best_params, f, indent=2)
print('Saved best params to', out_params_path)
pprint(best_params)


Saved best params to ..\4_data_analysis\model_datasets\best_params.json
{'rf_raw': {'max_depth': 10, 'max_features': 0.5, 'n_estimators': 200},
 'ridge_log1p': {'alpha': 10.0},
 'ridge_raw': {'alpha': 1.0},
 'xgb_log1p': {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 200}}


Notes and next steps
- This notebook ran a short randomized-style search using grouped expanding-window CV. You can extend n_iter and the param spaces for more exhaustive tuning.
- The CV function returns splits that ensure *no future data from any region is used for training* when evaluating validation folds.
- The saved best_params.json will be used in Notebook 5 for diagnostics/interpretability and Notebook 6 for final forecasts and model saving.

If any cell errors or you want me to change the CV window/step sizes, tell me the typical length of the time series per region and I'll adjust defaults and regenerate the notebook if desired.